# Dogs vs Cats Classification

In [14]:

import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [16]:

train_dir = 'data/train/'
test_dir = 'data/test/'


## Генераторы данных с аугментацией

In [17]:

img_size = 224
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    directory=os.path.dirname(test_dir),
    classes=[os.path.basename(test_dir)],
    target_size=(img_size, img_size),
    batch_size=1,
    class_mode=None,
    shuffle=False
)


Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


## Построение модели MobileNetV2

In [18]:

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_2[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

## Обучение модели

In [19]:

es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[es, checkpoint]
)


Epoch 1/10


2025-09-16 13:40:28.182022: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 51380224 exceeds 10% of free system memory.
2025-09-16 13:40:28.817508: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 51380224 exceeds 10% of free system memory.
2025-09-16 13:40:28.868611: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 154140672 exceeds 10% of free system memory.
2025-09-16 13:40:28.926680: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 156905472 exceeds 10% of free system memory.
2025-09-16 13:40:28.959822: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 38535168 exceeds 10% of free system memory.


625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 725ms/step - accuracy: 0.8132 - loss: 0.4197

625/625 ━━━━━━━━━━━━━━━━━━━━ 567s 899ms/step - accuracy: 0.8907 - loss: 0.2877 - val_accuracy: 0.9522 - val_loss: 0.1487
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 523ms/step - accuracy: 0.9460 - loss: 0.1505

625/625 ━━━━━━━━━━━━━━━━━━━━ 407s 651ms/step - accuracy: 0.9503 - loss: 0.1387 - val_accuracy: 0.9624 - val_loss: 0.1097
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 513ms/step - accuracy: 0.9571 - loss: 0.1141

...

625/625 ━━━━━━━━━━━━━━━━━━━━ 397s 636ms/step - accuracy: 0.9636 - loss: 0.0954 - val_accuracy: 0.9682 - val_loss: 0.0828
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 512ms/step - accuracy: 0.9636 - loss: 0.0922

625/625 ━━━━━━━━━━━━━━━━━━━━ 401s 641ms/step - accuracy: 0.9635 - loss: 0.0939 - val_accuracy: 0.9664 - val_loss: 0.0804
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 510ms/step - accuracy: 0.9633 - loss: 0.0918

625/625 ━━━━━━━━━━━━━━━━━━━━ 399s 638ms/step - accuracy: 0.9661 - loss: 0.0864 - val_accuracy: 0.9704 - val_loss: 0.0783
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 397s 636ms/step - accuracy: 0.9654 - loss: 0.0862 - val_accuracy: 0.9684 - val_loss: 0.0807
Epoch 9/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 511ms/step - accuracy: 0.9674 - loss: 0.0843

625/625 ━━━━━━━━━━━━━━━━━━━━ 400s 640ms/step - accuracy: 0.9667 - loss: 0.0859 - val_accuracy: 0.9704 - val_loss: 0.0760
Epoch 10/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 512ms/step - accuracy: 0.9657 - loss: 0.0880

625/625 ━━━━━━━━━━━━━━━━━━━━ 401s 641ms/step - accuracy: 0.9664 - loss: 0.0862 - val_accuracy: 0.9700 - val_loss: 0.0752


## Fine-tuning (размораживаю часть слоёв MobileNetV2)

In [20]:

for layer in base_model.layers[-40:]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

history_finetune = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    callbacks=[es, checkpoint]
)


Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 605ms/step - accuracy: 0.9025 - loss: 0.2331

625/625 ━━━━━━━━━━━━━━━━━━━━ 469s 737ms/step - accuracy: 0.9367 - loss: 0.1523 - val_accuracy: 0.9772 - val_loss: 0.0552
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 462s 738ms/step - accuracy: 0.9622 - loss: 0.0937 - val_accuracy: 0.9760 - val_loss: 0.0593
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 461s 738ms/step - accuracy: 0.9671 - loss: 0.0817 - val_accuracy: 0.9752 - val_loss: 0.0612
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 461s 738ms/step - accuracy: 0.9699 - loss: 0.0771 - val_accuracy: 0.9794 - val_loss: 0.0573


## Генерация сабмита для Kaggle

In [21]:

model.load_weights('best_model.h5')

preds = model.predict(test_generator, verbose=1)
submission = pd.DataFrame({
    'id': [os.path.splitext(os.path.basename(fname))[0] for fname in test_generator.filenames],
    'label': preds.ravel()
})

submission.to_csv('submission.csv', index=False)
submission.head()


12500/12500 ━━━━━━━━━━━━━━━━━━━━ 227s 18ms/step


,id,label
0,1,0.999988
1,10,0.000223
2,100,0.000781
3,1000,0.999984
4,10000,0.999989
